In [1]:
#Importando todos os módulos/bibliotecas
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import re
import win32com.client as win32
from datetime import datetime

In [2]:
options = webdriver.ChromeOptions()
#Argumento passado para o options que salva a contas e senhas que você abriu, no navegador de software automatizado.
options.add_argument(r'user-data-dir=C:\Users\joao_\AppData\Local\Google\Chrome\User Data\Profile Selenium')
#Argumento passado para o options para o navegador não detectar que está sendo controlado por um software automatizado
options.add_argument('--disable-blink-features=AutomationControlled')

#Abrindo o Navegador
service= Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=service, options=options)

In [3]:
#Importando a base de dados (data-frame), para filtrar as preferências com relação ao produto que será pesquisado
busca_precos = pd.read_excel("buscas.xlsx")

display(busca_precos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,placa de video rtx 3060,zota galax,4000,5500


In [4]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_termos_produtos(lista_termos_produtos, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_produtos:
        if not palavra in nome:
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos

#Function que faz a busca no google_shopping
def busca_google_shopping(navegador, produto, termos_banidos, preco_min, preco_max):
    produto = produto.casefold()
    termos_banidos = termos_banidos.casefold()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produtos = produto.split(" ")
    preco_min = float(preco_min)
    preco_max = float(preco_max)
    
    lista_ofertas = []
    #Entrar no google shopping
    
    #Abrindo a primeira aba, caso a minha lista de aba seja igual a 1.
    if len(navegador.window_handles)==1:
        navegador.get("https://www.google.com/")
        navegador.switch_to.window(navegador.window_handles[0])

    else:
        #Caso minha aba já exista, abrir ela de acordo com sua posição na lista de abas
        navegador.switch_to.window(navegador.window_handles[0])
    #Apaga caso tenha algo escrito
    if navegador.find_element(By.XPATH, '//*[@id="APjFqb"]').get_attribute("value") != None:
        navegador.find_element(By.XPATH, '//*[@id="APjFqb"]').clear()
    #Pesquisar o nome do produto
    time.sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto)
    navegador.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(Keys.ENTER)
    
    #Clicando na busca de pesquisa do google_shopping
    elementos = navegador.find_elements(By.CLASS_NAME, 'FMKtTb ')
    
    #Aqui utilizei um try e um except, pois o codigo estava dando um erro nessa parte.
    try:
        for item in elementos:
            if "Shopping" in item.text:
                item.click()
    except:
        pass
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__content')

    for resultado in lista_resultados:
        #Pegando o nome do produto
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nome = nome.casefold()

        #Verificando se o nome do produto possui os termos banidos ou não
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        #Verficiando se o nome do produto possui todos os termos desejados para a pesquisa
        tem_todos_termos_produtos = verificar_tem_termos_produtos(lista_termos_produtos, nome)

        if tem_termos_banidos==False and tem_todos_termos_produtos==True:
            #Pegando o valor do produto, e extraindo apenas o valor.
            preco = resultado.find_element(By.CLASS_NAME, 'XrAfOe').text
            #Extraindo apenas o valor utilizando a bibliteca regex
            try:
                padrao = re.compile('\d\.\d+,\d+')
                resultado_regex = re.findall(padrao, preco)
                preco_extraido = float(resultado_regex[0].replace('.', '').replace(',', '.'))
            except:
                continue
            #Verficando o produto está entre o valor de preco_min e preco_max, e extraindo o link
            if preco_min <= preco_extraido <= preco_max:
                 #Extraindo o link (usando um elemento como referencia, para conseguir extrair o link)
                elemento_referencia = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute('href')

                #Armazenando o resultado da busca em uma lista de tuplas
                lista_ofertas.append((nome, preco_extraido, link))
    return lista_ofertas

def busca_busca_pe(navegador, produto, termos_banidos, preco_min, preco_max):
    produto = produto.casefold()
    termos_banidos = termos_banidos.casefold()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produtos = produto.split(" ")
    preco_min = float(preco_min)
    preco_max = float(preco_max)
    
    lista_auxiliar = []
    
    #Abre uma nova aba e pesquisa no site do busca_pe diretamente se a minha lista de abas for igual a 1
    if len(navegador.window_handles)==1:
        navegador.execute_script(f"window.open('https://www.buscape.com.br/', '_blank')")
        navegador.switch_to.window(navegador.window_handles[1])
    else:
        #Abrindo a minha aba de índice 1 da minha lista de abas, caso o tamanho da lista for igual a 2
        navegador.switch_to.window(navegador.window_handles[1])
    time.sleep(1)    
    #Limpar a barra de pesquisa caso tenha algo escrito
    if navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').get_attribute("value") != None:
        for i in range(15):
            navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.BACKSPACE)
    #Pesquisando na barra de pesquisa
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto)
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.ENTER)
    
    #Pegando a lista de resultados da busca
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh')
    for resultado in lista_resultados:
        try:
            #Extraindo o nome do produto
            nome = resultado.find_element(By.CLASS_NAME, 'Text_Text__ARJdp ').text
            nome = nome.casefold()
        except:
            continue

        #Pegar apenas os produtos que não possuem os termos_banidos
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        #Pegar os produtos que possuem todos os termos desejados
        tem_todos_termos_produtos = verificar_tem_termos_produtos(lista_termos_produtos, nome)
        
        #Pegar os produtos que não iteração do  meu for seja: tem_termos_banidos==false e tem_todos_termos_produtos==true
        if tem_termos_banidos==False and tem_todos_termos_produtos==True:
            #Extraindo no preco do produto
            preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__HEz7L').text
            preco = preco.replace('R$', '').replace('.', '').replace(',', '.')
            preco = float(preco)
            if preco_min <= preco <= preco_max:
                #Extraindo pelo link
                elemento_referencia = resultado.find_element(By.CLASS_NAME, 'ProductCard_ProductCard_Tags___N1Ll')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute('href')
                
                #Armazenando o resultado da busca em uma lista de tuplas
                lista_auxiliar.append((nome, preco, link))
    return lista_auxiliar


In [5]:
# Percorrendo a base de dados, e salvando todos os resultados em apenas 1 dataframe.

tabela_ofertas = pd.DataFrame()

for linha in busca_precos.index:
    produto = busca_precos.loc[linha, 'Nome']
    termos_banidos = busca_precos.loc[linha, 'Termos banidos']
    preco_min = busca_precos.loc[linha, 'Preço mínimo']
    preco_max = busca_precos.loc[linha, 'Preço máximo']
    
    #Google Shopping
    lista_produtos_google_shopping = busca_google_shopping(navegador, produto, termos_banidos, preco_min, preco_max)
    if lista_produtos_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_produtos_google_shopping, columns=['Produto', 'Preço', 'Link de compra'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
        
    #Busca_pe
    lista_produtos_buscape = busca_busca_pe(navegador, produto, termos_banidos, preco_min, preco_max)
    if lista_produtos_buscape:
        tabela_buscape = pd.DataFrame(lista_produtos_buscape, columns=['Produto', 'Preço', 'Link de compra'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None

In [6]:
#Salvar a tabela de ofertas em uma planilha.
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)
tabelas_ofertas = pd.read_excel("Ofertas.xlsx")

display(tabelas_ofertas)

,Produto,Preço,Link de compra
0,"apple iphone 12 , 64gb - preto",3229.05,https://www.google.com/url?url=https://www.mer...
1,"iphone 12 64gb- branco, , color branco",3260.00,https://www.google.com/url?url=https://www.tra...
2,placa de video nvidia geforce rtx 3060 12 gb g...,5129.10,https://www.buscape.com.br/placa-de-video/plac...


In [14]:
def saudacao():
    saudacao = ''
    agora = datetime.now()
    if 4 <= agora.hour <= 12:
        saudacao = 'Bom dia'
    if 12 <= agora.hour <= 18:
        saudacao = 'Boa tarde'
    if 18 <= agora.hour <= 23:
        saudacao = 'Boa noite'
    return saudacao

#Enviar por e-mail a planilha de ofertas da busca dos resultados em formato HTML.

if len(tabela_ofertas) > 0:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'joao_cf55@hotmail.com'
    mail.Subject = 'Resultado da Pesquisa de preços'
    mail.HTMLBody = f'''
    <p>Prezados(a), {saudacao()}.</p>
    <p>Encontramos alguns produtos em oferta, dentro da faixa de preços desejada. Segue a tabela com detalhes</p>.
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou a disposição!</p>
    '''
    
    mail.Send()
    navegador.quit()